# TextCNN

In [33]:
from keras import Input, Model
from keras.layers import Embedding, Dense, Conv1D, GlobalMaxPooling1D, Concatenate, Dropout
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.preprocessing import sequence
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score

class TextCNN(object):
    def __init__(self, maxlen, max_features, embedding_dims,
                 class_num=1,
                 last_activation='sigmoid'):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))

        # Embedding part can try multichannel as same as origin paper
        embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)(input)
        convs = []
        for kernel_size in [3, 4, 5]:
            c = Conv1D(128, kernel_size, activation='relu')(embedding)
            c = GlobalMaxPooling1D()(c)
            convs.append(c)
        x = Concatenate()(convs)

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model

In [2]:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
epochs = 10

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = TextCNN(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=(x_test, y_test))

result = model.predict(x_test)
result[result>=0.5] = 1
result[result<0.5] = 0
acc = accuracy_score(result, y_test)
print('Test data accuracy is ', acc)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)...
x_train shape: (25000, 400)
x_test shape: (25000, 400)
Build model...
Instructions for updating:
Colocations handled automatically by placer.
Train...
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 47s 2ms/step - loss: 0.3916 - acc: 0.8142 - val_loss: 0.3195 - val_acc: 0.8627
Epoch 2/10
25000/25000 [==============================] - 46s 2ms/step - loss: 0.2114 - acc: 0.9160 - val_loss: 0.2531 - val_acc: 0.8962
Epoch 3/10
25000/25000 [==============================] - 49s 2ms/step - loss: 0.1331 - acc: 0.9524 - val_loss: 0.2729 - val_acc: 0.8920
Epoch 4/10
25000/25000 [==============================] - 48s 2ms/step - loss: 0.0758 - acc: 0.9768 - val_loss: 0.3136 - val_acc: 0.8871
Epoch 5/10
25000/25000 [========

# TextRNN

In [3]:
from keras import Input, Model
from keras.layers import Embedding, Dense, Dropout, LSTM
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.preprocessing import sequence

class TextRNN(object):
    def __init__(self, maxlen, max_features, embedding_dims,
                 class_num=1,
                 last_activation='sigmoid'):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))

        embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)(input)
        x = LSTM(128)(embedding)  # LSTM or GRU

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model

In [4]:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
epochs = 10

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = TextRNN(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=(x_test, y_test))

print('Test...')
result = model.predict(x_test)
result[result>=0.5] = 1
result[result<0.5] = 0
acc = accuracy_score(result, y_test)
print('Test data accuracy is ', acc)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)...
x_train shape: (25000, 400)
x_test shape: (25000, 400)
Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 147s 6ms/step - loss: 0.5209 - acc: 0.7399 - val_loss: 0.6029 - val_acc: 0.6649
Epoch 2/10
25000/25000 [==============================] - 159s 6ms/step - loss: 0.3440 - acc: 0.8560 - val_loss: 0.3223 - val_acc: 0.8609
Epoch 3/10
25000/25000 [==============================] - 156s 6ms/step - loss: 0.2670 - acc: 0.8942 - val_loss: 0.3262 - val_acc: 0.8650
Epoch 4/10
25000/25000 [==============================] - 152s 6ms/step - loss: 0.2238 - acc: 0.9134 - val_loss: 0.3200 - val_acc: 0.8655
Epoch 5/10
25000/25000 [==============================] - 154s 6ms/step - loss: 0.1927 - acc: 0.9266 - val_loss: 0.3469 - val_acc: 0.8701
Epoch 6/10
25000/25000 [==============================] - 153s 6ms/step - loss: 0.1725 -

# BiTextRNN

In [4]:
from keras import Input, Model
from keras.layers import Embedding, Dense, Dropout, LSTM, Bidirectional
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.preprocessing import sequence
from sklearn.metrics import accuracy_score

class BiTextRNN(object):
    def __init__(self, maxlen, max_features, embedding_dims,
                 class_num=1,
                 last_activation='sigmoid'):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))

        embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)(input)
        x = Bidirectional(LSTM(128))(embedding)  # LSTM or GRU

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model

In [7]:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
epochs = 10

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = BiTextRNN(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=(x_test, y_test))

print('Test...')
result = model.predict(x_test)
result[result>=0.5] = 1
result[result<0.5] = 0
acc = accuracy_score(result, y_test)
print('Test data accuracy is ', acc)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)...
x_train shape: (25000, 400)
x_test shape: (25000, 400)
Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 184s 7ms/step - loss: 0.6075 - acc: 0.6719 - val_loss: 0.6527 - val_acc: 0.6428
Epoch 2/10
25000/25000 [==============================] - 200s 8ms/step - loss: 0.4184 - acc: 0.8110 - val_loss: 0.3754 - val_acc: 0.8446
Epoch 3/10
25000/25000 [==============================] - 208s 8ms/step - loss: 0.3027 - acc: 0.8764 - val_loss: 0.3092 - val_acc: 0.8692
Epoch 4/10
25000/25000 [==============================] - 207s 8ms/step - loss: 0.2554 - acc: 0.8986 - val_loss: 0.3063 - val_acc: 0.8773
Epoch 5/10
25000/25000 [==============================] - 203s 8ms/step - loss: 0.2291 - acc: 0.9104 - val_loss: 0.2958 - val_acc: 0.8784
Epoch 6/10
25000/25000 [==============================] - 202s 8ms/step - loss: 0.1976 -

In [5]:
from keras import backend as K
from keras import initializers, regularizers, constraints
from keras.engine.topology import Layer


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2
            hidden = LSTM(64, return_sequences=True)(words)
            sentence = Attention()(hidden)
            # next add a Dense layer (for classification/regression) or whatever...
        """
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0

        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        e = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))  # e = K.dot(x, self.W)
        if self.bias:
            e += self.b
        e = K.tanh(e)

        a = K.exp(e)
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)

        c = K.sum(a * x, axis=1)
        return c

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.features_dim

In [6]:
from keras import Input, Model
from keras.layers import Embedding, Dense, Dropout, LSTM, Bidirectional
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.preprocessing import sequence
from sklearn.metrics import accuracy_score

class TextAttentionBiRNN(object):
    def __init__(self, maxlen, max_features, embedding_dims,
                 class_num=1,
                 last_activation='sigmoid'):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))

        embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)(input)
        x = Bidirectional(LSTM(128, return_sequences=True))(embedding)  # LSTM or GRU
        x = Attention(self.maxlen)(x)
        
        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model

In [13]:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
epochs = 10

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = TextAttentionBiRNN(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=(x_test, y_test))

print('Test...')
result = model.predict(x_test)
result[result>=0.5] = 1
result[result<0.5] = 0
acc = accuracy_score(result, y_test)
print('Test data accuracy is ', acc)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)...
x_train shape: (25000, 400)
x_test shape: (25000, 400)
Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 206s 8ms/step - loss: 0.4416 - acc: 0.7856 - val_loss: 0.3534 - val_acc: 0.8529
Epoch 2/10
25000/25000 [==============================] - 219s 9ms/step - loss: 0.2696 - acc: 0.8908 - val_loss: 0.2771 - val_acc: 0.8864
Epoch 3/10
25000/25000 [==============================] - 228s 9ms/step - loss: 0.2068 - acc: 0.9189 - val_loss: 0.2788 - val_acc: 0.8844
Epoch 4/10
25000/25000 [==============================] - 228s 9ms/step - loss: 0.1724 - acc: 0.9356 - val_loss: 0.2785 - val_acc: 0.8834
Epoch 5/10
25000/25000 [==============================] - 219s 9ms/step - loss: 0.1408 - acc: 0.9486 - val_loss: 0.2928 - val_acc: 0.8850
Test...
Test data accuracy is  0.88496


# RCNN (LSTM)

In [7]:
from keras import Input, Model
from keras import backend as K
from keras.layers import Embedding, Dense, SimpleRNN, Lambda, Concatenate, Conv1D, GlobalMaxPooling1D, LSTM
import numpy as np
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.preprocessing import sequence


class RCNN(object):
    def __init__(self, maxlen, max_features, embedding_dims,
                 class_num=1,
                 last_activation='sigmoid'):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input_current = Input((self.maxlen,))
        input_left = Input((self.maxlen,))
        input_right = Input((self.maxlen,))

        embedder = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)
        embedding_current = embedder(input_current)
        embedding_left = embedder(input_left)
        embedding_right = embedder(input_right)

        x_left = LSTM(128, return_sequences=True)(embedding_left)
        x_right = LSTM(128, return_sequences=True, go_backwards=True)(embedding_right)
        x_right = Lambda(lambda x: K.reverse(x, axes=1))(x_right)
        x = Concatenate(axis=2)([x_left, embedding_current, x_right])

        x = Conv1D(64, kernel_size=1, activation='tanh')(x)
        x = GlobalMaxPooling1D()(x)

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=[input_current, input_left, input_right], outputs=output)
        return model

In [17]:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
epochs = 10

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Prepare input for model...')
x_train_current = x_train
x_train_left = np.hstack([np.expand_dims(x_train[:, 0], axis=1), x_train[:, 0:-1]])
x_train_right = np.hstack([x_train[:, 1:], np.expand_dims(x_train[:, -1], axis=1)])
x_test_current = x_test
x_test_left = np.hstack([np.expand_dims(x_test[:, 0], axis=1), x_test[:, 0:-1]])
x_test_right = np.hstack([x_test[:, 1:], np.expand_dims(x_test[:, -1], axis=1)])
print('x_train_current shape:', x_train_current.shape)
print('x_train_left shape:', x_train_left.shape)
print('x_train_right shape:', x_train_right.shape)
print('x_test_current shape:', x_test_current.shape)
print('x_test_left shape:', x_test_left.shape)
print('x_test_right shape:', x_test_right.shape)

print('Build model...')
model = RCNN(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
model.fit([x_train_current, x_train_left, x_train_right], y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=([x_test_current, x_test_left, x_test_right], y_test))

print('Test...')
result = model.predict([x_test_current, x_test_left, x_test_right])
result[result>=0.5] = 1
result[result<0.5] = 0
acc = accuracy_score(result, y_test)
print('Test data accuracy is ', acc)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)...
x_train shape: (25000, 400)
x_test shape: (25000, 400)
Prepare input for model...
x_train_current shape: (25000, 400)
x_train_left shape: (25000, 400)
x_train_right shape: (25000, 400)
x_test_current shape: (25000, 400)
x_test_left shape: (25000, 400)
x_test_right shape: (25000, 400)
Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 224s 9ms/step - loss: 0.4024 - acc: 0.8178 - val_loss: 0.2848 - val_acc: 0.8806
Epoch 2/10
25000/25000 [==============================] - 229s 9ms/step - loss: 0.2363 - acc: 0.9056 - val_loss: 0.2576 - val_acc: 0.8965
Epoch 3/10
25000/25000 [==============================] - 228s 9ms/step - loss: 0.1699 - acc: 0.9348 - val_loss: 0.2542 - val_acc: 0.8980
Epoch 4/10
25000/25000 [==============================] - 232s 9ms/step - loss: 0.1238 - acc: 0.9558 - val_loss: 0.2962 - val_acc: 0.8

# RCNN (Attention + BiLstm)

In [8]:
from keras import Input, Model
from keras import backend as K
from keras.layers import Embedding, Dense, SimpleRNN, Lambda, Concatenate, Conv1D, GlobalMaxPooling1D, LSTM
import numpy as np
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras import initializers, regularizers, constraints
from keras.engine.topology import Layer


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2
            hidden = LSTM(64, return_sequences=True)(words)
            sentence = Attention()(hidden)
            # next add a Dense layer (for classification/regression) or whatever...
        """
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0

        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        e = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))  # e = K.dot(x, self.W)
        if self.bias:
            e += self.b
        e = K.tanh(e)

        a = K.exp(e)
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)

        c = K.sum(a * x, axis=1)
        return c

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.features_dim


class RCNN_Att_BiLstm(object):
    def __init__(self, maxlen, max_features, embedding_dims,
                 class_num=1,
                 last_activation='sigmoid'):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input_current = Input((self.maxlen,))
        input_left = Input((self.maxlen,))
        input_right = Input((self.maxlen,))

        embedder = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)
        embedding_current = embedder(input_current)
        embedding_left = embedder(input_left)
        embedding_right = embedder(input_right)
        
        x_left = Bidirectional(LSTM(128, return_sequences=True))(embedding_left)  # LSTM or GRU
        x_left = Attention(self.maxlen)(x_left)
        
        x_right = Bidirectional(LSTM(128, return_sequences=True))(embedding_right)  # LSTM or GRU
        x_right = Attention(self.maxlen)(x_right)

        #x_left = LSTM(128, return_sequences=True)(embedding_left)
        #x_right = LSTM(128, return_sequences=True, go_backwards=True)(embedding_right)
        x_right = Lambda(lambda x: K.reverse(x, axes=1))(x_right)
        
        x = Concatenate(axis=2)([x_left, embedding_current, x_right])

        x = Conv1D(64, kernel_size=1, activation='tanh')(x)
        x = GlobalMaxPooling1D()(x)

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=[input_current, input_left, input_right], outputs=output)
        return model

In [19]:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
epochs = 10

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Prepare input for model...')
x_train_current = x_train
x_train_left = np.hstack([np.expand_dims(x_train[:, 0], axis=1), x_train[:, 0:-1]])
x_train_right = np.hstack([x_train[:, 1:], np.expand_dims(x_train[:, -1], axis=1)])
x_test_current = x_test
x_test_left = np.hstack([np.expand_dims(x_test[:, 0], axis=1), x_test[:, 0:-1]])
x_test_right = np.hstack([x_test[:, 1:], np.expand_dims(x_test[:, -1], axis=1)])
print('x_train_current shape:', x_train_current.shape)
print('x_train_left shape:', x_train_left.shape)
print('x_train_right shape:', x_train_right.shape)
print('x_test_current shape:', x_test_current.shape)
print('x_test_left shape:', x_test_left.shape)
print('x_test_right shape:', x_test_right.shape)

print('Build model...')
model = RCNN(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
model.fit([x_train_current, x_train_left, x_train_right], y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=([x_test_current, x_test_left, x_test_right], y_test))

print('Test...')
result = model.predict([x_test_current, x_test_left, x_test_right])
result[result>=0.5] = 1
result[result<0.5] = 0
acc = accuracy_score(result, y_test)
print('Test data accuracy is ', acc)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)...
x_train shape: (25000, 400)
x_test shape: (25000, 400)
Prepare input for model...
x_train_current shape: (25000, 400)
x_train_left shape: (25000, 400)
x_train_right shape: (25000, 400)
x_test_current shape: (25000, 400)
x_test_left shape: (25000, 400)
x_test_right shape: (25000, 400)
Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 222s 9ms/step - loss: 0.3931 - acc: 0.8187 - val_loss: 0.2986 - val_acc: 0.8729
Epoch 2/10
25000/25000 [==============================] - 230s 9ms/step - loss: 0.2336 - acc: 0.9054 - val_loss: 0.2669 - val_acc: 0.8896
Epoch 3/10
25000/25000 [==============================] - 232s 9ms/step - loss: 0.1691 - acc: 0.9364 - val_loss: 0.2684 - val_acc: 0.8916
Epoch 4/10
25000/25000 [==============================] - 237s 9ms/step - loss: 0.1163 - acc: 0.9596 - val_loss: 0.3161 - val_acc: 0.8

# RCNNVariant

In [9]:
from keras import Input, Model
from keras.layers import Embedding, Dense, Concatenate, Conv1D, Bidirectional, LSTM, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.preprocessing import sequence

class RCNNVariant(object):
    """Variant of RCNN.
        Base on structure of RCNN, we do some improvement:
        1. Ignore the shift for left/right context.
        2. Use Bidirectional LSTM/GRU to encode context.
        3. Use Multi-CNN to represent the semantic vectors.
        4. Use ReLU instead of Tanh.
        5. Use both AveragePooling and MaxPooling.
    """

    def __init__(self, maxlen, max_features, embedding_dims,
                 class_num=1,
                 last_activation='sigmoid'):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))

        embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)(input)

        x_context = Bidirectional(LSTM(128, return_sequences=True))(embedding)
        x = Concatenate()([embedding, x_context])

        convs = []
        for kernel_size in range(1, 5):
            conv = Conv1D(128, kernel_size, activation='relu')(x)
            convs.append(conv)
        poolings = [GlobalAveragePooling1D()(conv) for conv in convs] + [GlobalMaxPooling1D()(conv) for conv in convs]
        x = Concatenate()(poolings)

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model

In [38]:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
epochs = 10

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = RCNNVariant(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=(x_test, y_test))

print('Test...')
result = model.predict(x_test)
result[result>=0.5] = 1
result[result<0.5] = 0
acc = accuracy_score(result, y_test)
print('Test data accuracy is ', acc)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)...
x_train shape: (25000, 400)
x_test shape: (25000, 400)
Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 389s 16ms/step - loss: 0.3569 - acc: 0.8370 - val_loss: 0.2580 - val_acc: 0.8932
Epoch 2/10
25000/25000 [==============================] - 397s 16ms/step - loss: 0.2030 - acc: 0.9220 - val_loss: 0.2470 - val_acc: 0.8983
Epoch 3/10
25000/25000 [==============================] - 385s 15ms/step - loss: 0.1520 - acc: 0.9412 - val_loss: 0.2626 - val_acc: 0.8954
Epoch 4/10
25000/25000 [==============================] - 389s 16ms/step - loss: 0.1014 - acc: 0.9647 - val_loss: 0.3091 - val_acc: 0.8868
Epoch 5/10
25000/25000 [==============================] - 386s 15ms/step - loss: 0.0616 - acc: 0.9792 - val_loss: 0.4233 - val_acc: 0.8780
Test...
Test data accuracy is  0.87796


# HA

In [10]:
from keras import backend as K
from keras import initializers, regularizers, constraints
from keras.engine.topology import Layer
from keras import Input, Model
from keras.layers import Embedding, Dense, Bidirectional, LSTM, TimeDistributed
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.preprocessing import sequence


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2
            hidden = LSTM(64, return_sequences=True)(words)
            sentence = Attention()(hidden)
            # next add a Dense layer (for classification/regression) or whatever...
        """
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0

        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        e = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))  # e = K.dot(x, self.W)
        if self.bias:
            e += self.b
        e = K.tanh(e)

        a = K.exp(e)
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)

        c = K.sum(a * x, axis=1)
        return c

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.features_dim
    
class HAN(object):
    def __init__(self, maxlen_sentence, maxlen_word, max_features, embedding_dims,
                 class_num=1,
                 last_activation='sigmoid'):
        self.maxlen_sentence = maxlen_sentence
        self.maxlen_word = maxlen_word
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        # Word part
        input_word = Input(shape=(self.maxlen_word,))
        x_word = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen_word)(input_word)
        x_word = Bidirectional(LSTM(128, return_sequences=True))(x_word)  # LSTM or GRU
        x_word = Attention(self.maxlen_word)(x_word)
        model_word = Model(input_word, x_word)

        # Sentence part
        input = Input(shape=(self.maxlen_sentence, self.maxlen_word))
        x_sentence = TimeDistributed(model_word)(input)
        x_sentence = Bidirectional(LSTM(128, return_sequences=True))(x_sentence)  # LSTM or GRU
        x_sentence = Attention(self.maxlen_sentence)(x_sentence)

        output = Dense(self.class_num, activation=self.last_activation)(x_sentence)
        model = Model(inputs=input, outputs=output)
        return model

In [31]:
max_features = 5000
maxlen_sentence = 16
maxlen_word = 25
batch_size = 32
embedding_dims = 50
epochs = 10

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x #sentence x #word)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen_sentence * maxlen_word)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen_sentence * maxlen_word)
x_train = x_train.reshape((len(x_train), maxlen_sentence, maxlen_word))
x_test = x_test.reshape((len(x_test), maxlen_sentence, maxlen_word))
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = HAN(maxlen_sentence, maxlen_word, max_features, embedding_dims).get_model()
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=(x_test, y_test))

print('Test...')
result = model.predict(x_test)
result[result>=0.5] = 1
result[result<0.5] = 0
acc = accuracy_score(result, y_test)
print('Test data accuracy is ', acc)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x #sentence x #word)...
x_train shape: (25000, 16, 25)
x_test shape: (25000, 16, 25)
Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 160s 6ms/step - loss: 0.3617 - acc: 0.8330 - val_loss: 0.3229 - val_acc: 0.8607
Epoch 2/10
25000/25000 [==============================] - 164s 7ms/step - loss: 0.2338 - acc: 0.9078 - val_loss: 0.2705 - val_acc: 0.8865
Epoch 3/10
25000/25000 [==============================] - 172s 7ms/step - loss: 0.1882 - acc: 0.9293 - val_loss: 0.2873 - val_acc: 0.8844
Epoch 4/10
25000/25000 [==============================] - 171s 7ms/step - loss: 0.1501 - acc: 0.9457 - val_loss: 0.2986 - val_acc: 0.8859
Epoch 5/10
25000/25000 [==============================] - 171s 7ms/step - loss: 0.1141 - acc: 0.9608 - val_loss: 0.3329 - val_acc: 0.8839
Test...
Test data accuracy is  0.88392


# FastText

In [11]:
from keras import Input, Model
from keras.layers import Embedding, GlobalAveragePooling1D, Dense
import numpy as np
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.preprocessing import sequence


class FastText(object):
    def __init__(self, maxlen, max_features, embedding_dims,
                 class_num=1,
                 last_activation='sigmoid'):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))

        embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)(input)
        x = GlobalAveragePooling1D()(embedding)

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model

In [36]:
def create_ngram_set(input_list, ngram_value=2):
    """
    Extract a set of n-grams from a list of integers.
    # >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=2)
    {(4, 9), (4, 1), (1, 4), (9, 4)}
    # >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=3)
    [(1, 4, 9), (4, 9, 4), (9, 4, 1), (4, 1, 4)]
    """
    return set(zip(*[input_list[i:] for i in range(ngram_value)]))


def add_ngram(sequences, token_indice, ngram_range=2):
    """
    Augment the input list of list (sequences) by appending n-grams values.
    Example: adding bi-gram
    # >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    # >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017}
    # >>> add_ngram(sequences, token_indice, ngram_range=2)
    [[1, 3, 4, 5, 1337, 2017], [1, 3, 7, 9, 2, 1337, 42]]
    Example: adding tri-gram
    # >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    # >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017, (7, 9, 2): 2018}
    # >>> add_ngram(sequences, token_indice, ngram_range=3)
    [[1, 3, 4, 5, 1337, 2017], [1, 3, 7, 9, 2, 1337, 42, 2018]]
    """
    new_sequences = []
    for input_list in sequences:
        new_list = input_list[:]
        for ngram_value in range(2, ngram_range + 1):
            for i in range(len(new_list) - ngram_value + 1):
                ngram = tuple(new_list[i:i + ngram_value])
                if ngram in token_indice:
                    new_list.append(token_indice[ngram])
        new_sequences.append(new_list)

    return new_sequences


# Set parameters:
# ngram_range = 2 will add bi-grams features
ngram_range = 1
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
epochs = 50

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Average train sequence length: {}'.format(np.mean(list(map(len, x_train)), dtype=int)))
print('Average test sequence length: {}'.format(np.mean(list(map(len, x_test)), dtype=int)))

if ngram_range > 1:
    print('Adding {}-gram features'.format(ngram_range))
    # Create set of unique n-gram from the training set.
    ngram_set = set()
    for input_list in x_train:
        for i in range(2, ngram_range + 1):
            set_of_ngram = create_ngram_set(input_list, ngram_value=i)
            ngram_set.update(set_of_ngram)

    # Dictionary mapping n-gram token to a unique integer.
    # Integer values are greater than max_features in order
    # to avoid collision with existing features.
    start_index = max_features + 1
    token_indice = {v: k + start_index for k, v in enumerate(ngram_set)}
    indice_token = {token_indice[k]: k for k in token_indice}

    # max_features is the highest integer that could be found in the dataset.
    max_features = np.max(list(indice_token.keys())) + 1

    # Augmenting x_train and x_test with n-grams features
    x_train = add_ngram(x_train, token_indice, ngram_range)
    x_test = add_ngram(x_test, token_indice, ngram_range)
    print('Average train sequence length: {}'.format(np.mean(list(map(len, x_train)), dtype=int)))
    print('Average test sequence length: {}'.format(np.mean(list(map(len, x_test)), dtype=int)))

print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = FastText(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=(x_test, y_test))

print('Test...')
result = model.predict(x_test)
result[result>=0.5] = 1
result[result<0.5] = 0
acc = accuracy_score(result, y_test)
print('Test data accuracy is ', acc)

Loading data...
25000 train sequences
25000 test sequences
Average train sequence length: 238
Average test sequence length: 230
Pad sequences (samples x time)...
x_train shape: (25000, 400)
x_test shape: (25000, 400)
Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/50
25000/25000 [==============================] - 4s 141us/step - loss: 0.6170 - acc: 0.7280 - val_loss: 0.5125 - val_acc: 0.8145
Epoch 2/50
25000/25000 [==============================] - 3s 115us/step - loss: 0.4228 - acc: 0.8535 - val_loss: 0.3802 - val_acc: 0.8615
Epoch 3/50
25000/25000 [==============================] - 3s 116us/step - loss: 0.3311 - acc: 0.8796 - val_loss: 0.3284 - val_acc: 0.8745
Epoch 4/50
25000/25000 [==============================] - 3s 118us/step - loss: 0.2885 - acc: 0.8924 - val_loss: 0.3042 - val_acc: 0.8827
Epoch 5/50
25000/25000 [==============================] - 3s 115us/step - loss: 0.2621 - acc: 0.9028 - val_loss: 0.2909 - val_acc: 0.8844
Epoch 6/50
25000/25

In [21]:
from sklearn.model_selection import train_test_split
import jieba
from keras.preprocessing.text import Tokenizer

def preprocessing(data):
    stopword_list = [k.strip() for k in open('./data/stop_words.utf8', encoding='utf8').readlines() if k.strip() != '']
    # 处理data
    cutWords_data = []
    for article in data['body'].astype(str):
        cutWords = [k for k in jieba.cut(article) if k not in stopword_list]
        cutWords_data.append(cutWords)
    tokenizer = Tokenizer(num_words=1000)  # 建立一个2000个单词的字典
    tokenizer.fit_on_texts(cutWords_data)
    # 对每一篇文章转换为数字列表，使用每个词的编号进行编号
    x_data_seq = tokenizer.texts_to_sequences(cutWords_data)
    x_data = sequence.pad_sequences(x_data_seq, maxlen=400)
    y_data = np.array(data['label'].values)

    x_train, x_test, y_train,  y_test = train_test_split(x_data, y_data,test_size=0.2, random_state=42)
    
    return x_train, y_train, x_test, y_test

# 水文标注预测

In [22]:
import pandas as pd

data = pd.read_csv('./data/water_artical.tsv', sep='\t')

In [23]:
data.head()

,a.id,label,url,imgnum,title,body
0,ffbe23538f7c9c15b1fb3bce338d39f1,1,https://renderh5ui.inveno.com/detail.html?app=...,0,“懂事的女孩都不要彩礼”“婆婆，那不是懂事，那是没脑子”,用真诚易懂的话语，讲述关于养生的小知识，大家好，这里是养生小博士，感谢大家的观看，希望大...
1,86990225caf5829bc4b5f7b145a7e2ac,0,https://youliao.163yun.com/api-server/rss/xiao...,0,黄毅清曝崔永元因被调查畏罪自杀？崔永元最新回应证明自己安全,3月1号凌晨，黄毅清突然在社交平台中发文称“崔永元自杀了”，引起舆论一片哗然，不知道是否真...
2,fed8e7c9424115e07ce4860848440582,0,https://youliao.163yun.com/api-server/rss/xiao...,0,男子双11网购了一箱方便面，收到快递后，打开包裹直接吐了,各位网友大家好，所谓生活瞬息万变，今天的我们永远无法预测明天会发生什么，世界之大每天都会有不...
3,f8911a9e254d4fc2813d578ad8cd1cef,1,https://renderh5ui.inveno.com/detail.html?app=...,0,老婆来月事冲我发火，我对她说了一句话，第二天小姨子上门来骂我,现在女人真是很麻烦，其实我们男人很清楚，她们女人每一个月来这种事情很麻烦，但是...
4,9c4bc77498ec876614cb47bfd8edb978,0,https://xw.qq.com/partner/mibrowser/20190118A0...,3,权健风波未平，无限极又被推上风口浪尖,2019年1月1日，天津市公安机关对权健公司涉嫌组织、领导传销活动罪和虚假广告罪立案侦查。最...


In [81]:
x_train, y_train, x_test, y_test = preprocessing(data)

In [31]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (7776, 400)
x_test shape: (1945, 400)


In [36]:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
epochs = 10

model = TextCNN(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=(x_test, y_test))

print('Test...')
result = model.predict(x_test)
auc = roc_auc_score(y_test, result)

result[result>=0.5] = 1
result[result<0.5] = 0

acc = accuracy_score(y_test, result)
precise =  precision_score(y_test, result)
recall = recall_score(y_test, result)

print('accuracy is ', acc)
print('precise is ', precise)
print('recall is ', recall)
print('auc is ', auc)

Train...
Train on 7776 samples, validate on 1945 samples
Epoch 1/10
7776/7776 [==============================] - 13s 2ms/step - loss: 0.4993 - acc: 0.7388 - val_loss: 0.4582 - val_acc: 0.7784
Epoch 2/10
7776/7776 [==============================] - 13s 2ms/step - loss: 0.3788 - acc: 0.8207 - val_loss: 0.4384 - val_acc: 0.7825
Epoch 3/10
7776/7776 [==============================] - 13s 2ms/step - loss: 0.3368 - acc: 0.8476 - val_loss: 0.4272 - val_acc: 0.7964
Epoch 4/10
7776/7776 [==============================] - 14s 2ms/step - loss: 0.2960 - acc: 0.8708 - val_loss: 0.4402 - val_acc: 0.7943
Epoch 5/10
7776/7776 [==============================] - 13s 2ms/step - loss: 0.2543 - acc: 0.8933 - val_loss: 0.4631 - val_acc: 0.7959
Epoch 6/10
7776/7776 [==============================] - 14s 2ms/step - loss: 0.2066 - acc: 0.9192 - val_loss: 0.4922 - val_acc: 0.7933
accuracy is  0.7933161953727507
precise is  0.8228279386712095
recall is  0.8327586206896552
auc is  0.8783999560729189


In [37]:
model = RCNNVariant(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=(x_test, y_test))

print('Test...')
result = model.predict(x_test)
auc = roc_auc_score(y_test, result)

result[result>=0.5] = 1
result[result<0.5] = 0

acc = accuracy_score(y_test, result)
precise =  precision_score(y_test, result)
recall = recall_score(y_test, result)

print('accuracy is ', acc)
print('precise is ', precise)
print('recall is ', recall)
print('auc is ', auc)

Train...
Train on 7776 samples, validate on 1945 samples
Epoch 1/10
7776/7776 [==============================] - 103s 13ms/step - loss: 0.4670 - acc: 0.7654 - val_loss: 0.4370 - val_acc: 0.7871
Epoch 2/10
7776/7776 [==============================] - 108s 14ms/step - loss: 0.3864 - acc: 0.8207 - val_loss: 0.4229 - val_acc: 0.7943
Epoch 3/10
7776/7776 [==============================] - 108s 14ms/step - loss: 0.3551 - acc: 0.8427 - val_loss: 0.4346 - val_acc: 0.7923
Epoch 4/10
7776/7776 [==============================] - 111s 14ms/step - loss: 0.3213 - acc: 0.8593 - val_loss: 0.4531 - val_acc: 0.7820
Epoch 5/10
7776/7776 [==============================] - 109s 14ms/step - loss: 0.2820 - acc: 0.8813 - val_loss: 0.5235 - val_acc: 0.7805
Test...
accuracy is  0.7804627249357327
precise is  0.7749437359339835
recall is  0.8905172413793103
auc is  0.8758873270371184


# RCNN(ATTENTION+LSTM)

In [39]:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
epochs = 3

print('Prepare input for model...')
x_train_current = x_train
x_train_left = np.hstack([np.expand_dims(x_train[:, 0], axis=1), x_train[:, 0:-1]])
x_train_right = np.hstack([x_train[:, 1:], np.expand_dims(x_train[:, -1], axis=1)])
x_test_current = x_test
x_test_left = np.hstack([np.expand_dims(x_test[:, 0], axis=1), x_test[:, 0:-1]])
x_test_right = np.hstack([x_test[:, 1:], np.expand_dims(x_test[:, -1], axis=1)])
print('x_train_current shape:', x_train_current.shape)
print('x_train_left shape:', x_train_left.shape)
print('x_train_right shape:', x_train_right.shape)
print('x_test_current shape:', x_test_current.shape)
print('x_test_left shape:', x_test_left.shape)
print('x_test_right shape:', x_test_right.shape)

print('Build model...')
model = RCNN(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
model.fit([x_train_current, x_train_left, x_train_right], y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=([x_test_current, x_test_left, x_test_right], y_test))

print('Test...')
result = model.predict([x_test_current, x_test_left, x_test_right])
auc = roc_auc_score(y_test, result)

result[result>=0.5] = 1
result[result<0.5] = 0

acc = accuracy_score(y_test, result)
precise =  precision_score(y_test, result)
recall = recall_score(y_test, result)

print('accuracy is ', acc)
print('precise is ', precise)
print('recall is ', recall)
print('auc is ', auc)

Prepare input for model...
x_train_current shape: (7776, 400)
x_train_left shape: (7776, 400)
x_train_right shape: (7776, 400)
x_test_current shape: (1945, 400)
x_test_left shape: (1945, 400)
x_test_right shape: (1945, 400)
Build model...
Train...
Train on 7776 samples, validate on 1945 samples
Epoch 1/3
7776/7776 [==============================] - 57s 7ms/step - loss: 0.4958 - acc: 0.7559 - val_loss: 0.4407 - val_acc: 0.7897
Epoch 2/3
7776/7776 [==============================] - 59s 8ms/step - loss: 0.3891 - acc: 0.8239 - val_loss: 0.4247 - val_acc: 0.7902
Epoch 3/3
7776/7776 [==============================] - 60s 8ms/step - loss: 0.3603 - acc: 0.8421 - val_loss: 0.4249 - val_acc: 0.7943
Test...
accuracy is  0.794344473007712
precise is  0.8166666666666667
recall is  0.8448275862068966
auc is  0.8822457720184493


In [43]:
max_features = 5000
maxlen_sentence = 16
maxlen_word = 25
batch_size = 32
embedding_dims = 50
epochs = 3

x_train = x_train.reshape((len(x_train), maxlen_sentence, maxlen_word))
x_test = x_test.reshape((len(x_test), maxlen_sentence, maxlen_word))
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

model = HAN(maxlen_sentence, maxlen_word, max_features, embedding_dims).get_model()
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=(x_test, y_test))

print('Test...')
result = model.predict(x_test)
auc = roc_auc_score(y_test, result)

result[result>=0.5] = 1
result[result<0.5] = 0

acc = accuracy_score(y_test, result)
precise =  precision_score(y_test, result)
recall = recall_score(y_test, result)

print('accuracy is ', acc)
print('precise is ', precise)
print('recall is ', recall)
print('auc is ', auc)

x_train shape: (7776, 16, 25)
x_test shape: (1945, 16, 25)
Train...
Train on 7776 samples, validate on 1945 samples
Epoch 1/3
7776/7776 [==============================] - 47s 6ms/step - loss: 0.4938 - acc: 0.7423 - val_loss: 0.4324 - val_acc: 0.7933
Epoch 2/3
7776/7776 [==============================] - 44s 6ms/step - loss: 0.3923 - acc: 0.8192 - val_loss: 0.4241 - val_acc: 0.8036
Epoch 3/3
7776/7776 [==============================] - 45s 6ms/step - loss: 0.3712 - acc: 0.8301 - val_loss: 0.4495 - val_acc: 0.7887
Test...
accuracy is  0.7886889460154242
precise is  0.7826415094339623
recall is  0.8939655172413793
auc is  0.8845607291895454


In [44]:
data_embed = pd.read_csv('./data/data_sent.csv', sep='\t')
data_embed.head()

,a.id,label,url,imgnum,title,body,title_sent2Vec
0,ffbe23538f7c9c15b1fb3bce338d39f1,1,https://renderh5ui.inveno.com/detail.html?app=...,0,“懂事的女孩都不要彩礼”“婆婆，那不是懂事，那是没脑子”,用真诚易懂的话语，讲述关于养生的小知识，大家好，这里是养生小博士，感谢大家的观看，希望大...,[ 0.06967958 0.17589551 -0.01697676 0.072335...
1,86990225caf5829bc4b5f7b145a7e2ac,0,https://youliao.163yun.com/api-server/rss/xiao...,0,黄毅清曝崔永元因被调查畏罪自杀？崔永元最新回应证明自己安全,3月1号凌晨，黄毅清突然在社交平台中发文称“崔永元自杀了”，引起舆论一片哗然，不知道是否真...,[-0.04066085 0.22784638 -0.09804465 -0.159352...
2,fed8e7c9424115e07ce4860848440582,0,https://youliao.163yun.com/api-server/rss/xiao...,0,男子双11网购了一箱方便面，收到快递后，打开包裹直接吐了,各位网友大家好，所谓生活瞬息万变，今天的我们永远无法预测明天会发生什么，世界之大每天都会有不...,[ 1.78375721e-01 1.86844319e-01 -1.13675095e-...
3,f8911a9e254d4fc2813d578ad8cd1cef,1,https://renderh5ui.inveno.com/detail.html?app=...,0,老婆来月事冲我发火，我对她说了一句话，第二天小姨子上门来骂我,现在女人真是很麻烦，其实我们男人很清楚，她们女人每一个月来这种事情很麻烦，但是...,[-0.00909939 0.10430206 -0.12136529 -0.030800...
4,9c4bc77498ec876614cb47bfd8edb978,0,https://xw.qq.com/partner/mibrowser/20190118A0...,3,权健风波未平，无限极又被推上风口浪尖,2019年1月1日，天津市公安机关对权健公司涉嫌组织、领导传销活动罪和虚假广告罪立案侦查。最...,[-0.05588765 0.3731641 -0.21718842 -0.304607...


In [73]:
def preprocessing1(data):
    # 处理data
    cutWords_data = []
    for article in data['title_sent2Vec'].astype(str):
        vector_str = article.replace('[', '')
        vector_str = vector_str.replace(']', '').strip()
        vector_str = vector_str.split(" ")
        vector = []
        for i in range(0, len(vector_str)):
            if vector_str[i] != '':
                if vector_str[i].endswith('\n'):
                    vector.append(vector_str[i].replace('\n', ''))
                else:
                    vector.append(vector_str[i])
        
        
        cutWords_data.append(vector)
  
    x_data = np.array(cutWords_data)
    y_data = np.array(data['label'].values)

    x_train, x_test, y_train,  y_test = train_test_split(x_data, y_data,test_size=0.2, random_state=42)
    
    return x_train, y_train, x_test, y_test